In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

### Normalize the Data and Separate Patients into Individual Files

In [2]:
# Load Datasets
uc = pd.read_csv('/Users/emmadyer/Desktop/ibd_long_project/data/uc_expanded.csv')
cd = pd.read_csv('/Users/emmadyer/Desktop/ibd_long_project/data/cd_expanded.csv')
healthy = pd.read_csv('/Users/emmadyer/Desktop/ibd_long_project/data/healthy_expanded.csv')
test_ibd = pd.read_csv('/Users/emmadyer/Desktop/ibd_long_project/data/test_ibd_expanded.csv')

In [3]:
# Make a list of the lab values (column headers)
lab_value_names = list(healthy.columns.values)
lab_value_names = lab_value_names[1:-1]

# Make a list of the DataFrames
all_dfs = [uc, cd, healthy]

In [4]:
# Normalize Each Patient's Data
# Min/Max scaling [0,1] and account for missing values
def minmax_scale(x, min, max):
    ''' 
    Min/Max Scaler [0,1] that also handles when the max=min.
    Inputs:
    x (int) - Value to be normalized
    min (int) - minmum value
    max (int) - maximum value

    Outputs:
    norm (int) - normalized value [0,1]
    '''
    if min != max:
        scaled = max - min
    else:
        scaled = 1
    norm = (x - min) / scaled
    return norm

In [5]:
dataset_norms = []
for df in all_dfs:
    pts_lst = []
    mean_norm_values = []
    for lab in lab_value_names:
        #print("Lab Name: ", lab)
        values = df[lab].fillna(0)
        #values = values.astype(np.float16)
        min_lab = min(values)
        #print("Minimum Value: ", min_lab)
        max_lab = max(values)
        #print("Maximum Value: ", max_lab)
        mean_value = values.mean()
        norm_lab_val = minmax_scale(mean_value, min_lab, max_lab)
        #print('Scaled Value: ', norm_lab_val, '\n')
        mean_norm_values.append(norm_lab_val)
    dataset_norms.append(mean_norm_values)

In [6]:
# Need to calculate a mean normalized value in the event that a given patient only has one recorded value or no values.
# Mean Noramlized Values

# Lists to store patient normalized dataframes
all_patients_lsts = []
file_names = ['uc', 'cd', 'healthy']
for pt_population, df in enumerate(all_dfs):
    mean_norm_values = dataset_norms[pt_population]
    patient_id_lst = list(df.patient_id.unique())
    print(len(patient_id_lst))
    for patient in patient_id_lst:
        patient_df = df[df['patient_id'] == patient].reset_index()
        # Patient Normalized Values
        visits = list(range(0,5))
        na_reference_df = patient_df.copy()
        na_reference_df = na_reference_df.isnull()
        #norm_df = patient_df.copy().reset_index()
        for i, lab in enumerate(lab_value_names):
            #print("\n Lab Name: ", lab)
            total_missing = patient_df[lab].isnull().sum()
            if total_missing == 5: 
                continue
            else:
                values = patient_df[lab].fillna(0)
                min_lab = min(values)
                max_lab = max(values)
                #print("Minimum Value: ", min_lab)
                #print("Maximum Value: ", max_lab)
                for v in visits:
                    is_missing = na_reference_df.loc[v, lab]
                    #print("Missing???: ", is_missing)
                    if is_missing == False:
                        value = patient_df.loc[v, lab]
                        norm_val = minmax_scale(value, min_lab, max_lab)
                        patient_df.loc[v, lab] = norm_val
                    #print("Lab Value: ", value)
                    #print('Scaled Value: ', norm_val)
        pts_lst.append(patient_df)
        file_path = '/Users/emmadyer/Desktop/ibd_long_project/data/' + file_names[pt_population] + '_pts/' + str(patient) + '.csv'
        patient_df.to_csv(file_path, index=False)
        #print(norm_df)
    all_patients_lsts.append(pts_lst)
        

1688
2197
5415


### Calculate Distance Correlation
This is calculated on the whole dataset. We are solving for the distance between two vectors. In this case, we are calculating the distance correlation for the pairwise vectors for each analyte. 